Portfolio Suitability

In [1]:
#Imports

import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from fpdf import FPDF
# from MCForecastTools import MCSimulation
from pathlib import Path
%matplotlib inline
# from mailer import Mailer
import questionary
import fire
from questionary.constants import NO, YES, YES_OR_NO
import sqlalchemy


In [2]:
# create table

database_connection_string = "sqlite:///"
engine = sqlalchemy.create_engine(database_connection_string, echo=True)

engine.table_names()

create_table = (

"""
CREATE TABLE client(
    "full_name" VARCHAR,
    "phone_number" VARCHAR,
    "email_address" VARCHAR,
    "annual_income" FLOAT,
    "investing_experience" FLOAT,
    "investment_amount" FLOAT,
    "annual_expenses" FLOAT,
    "income_stability" VARCHAR,
    "risk_level" VARCHAR,
    "investment_strategy" VARCHAR,
    "investment_length" VARCHAR
)
"""
)
engine.execute(create_table)


2022-02-07 20:44:51,748 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-02-07 20:44:51,749 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-07 20:44:51,749 INFO sqlalchemy.engine.Engine 
CREATE TABLE client(
    "full_name" VARCHAR,
    "phone_number" VARCHAR,
    "email_address" VARCHAR,
    "annual_income" FLOAT,
    "investing_experience" FLOAT,
    "investment_amount" FLOAT,
    "annual_expenses" FLOAT,
    "income_stability" VARCHAR,
    "risk_level" VARCHAR,
    "investment_strategy" VARCHAR,
    "investment_length" VARCHAR
)

2022-02-07 20:44:51,750 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-07 20:44:51,751 INFO sqlalchemy.engine.Engine COMMIT


/var/folders/4x/jx46p2kn29bbg4k41vh6c4gh0000gn/T/ipykernel_16616/665304045.py:6: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


In [3]:
# Load dummy data for testing only.  DELETE BEFORE USING

full_name = "Jacob"
phone_number = "713-555-7834"
email_address = "cdb@bcg.law"
annual_income = 100000
investing_experience = 4
investment_amount = 10000
annual_expenses = 50000
income_stability = "YES"
risk_level = "Low"
investment_strategy = "Value"
investment_length = "0-1"

insert_data = """
    INSERT INTO client (
        'full_name', 'phone_number', 'email_address', 'annual_income', 'investing_experience', 'investment_amount', 'annual_expenses', 'income_stability', 'risk_level', 
        'investment_strategy', 'investment_length')
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """
    
engine.execute(insert_data, (full_name, phone_number, email_address, annual_income, investing_experience, investment_amount, annual_expenses, income_stability, risk_level, investment_strategy, investment_length))
    

2022-02-07 20:44:53,869 INFO sqlalchemy.engine.Engine 
    INSERT INTO client (
        'full_name', 'phone_number', 'email_address', 'annual_income', 'investing_experience', 'investment_amount', 'annual_expenses', 'income_stability', 'risk_level', 
        'investment_strategy', 'investment_length')
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    
2022-02-07 20:44:53,874 INFO sqlalchemy.engine.Engine [raw sql] ('Jacob', '713-555-7834', 'cdb@bcg.law', 100000, 4, 10000, 50000, 'YES', 'Low', 'Value', '0-1')
2022-02-07 20:44:53,880 INFO sqlalchemy.engine.Engine COMMIT


In [10]:
engine.table_names(client)

read_all_data = """
SELECT * FROM client
"""
results = engine.execute(read_all_data)
for row in results:
    print(row)


2022-02-07 20:51:32,923 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-02-07 20:51:32,926 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-07 20:51:32,928 INFO sqlalchemy.engine.Engine 
SELECT * FROM client

2022-02-07 20:51:32,930 INFO sqlalchemy.engine.Engine [raw sql] ()
Jacob


/var/folders/4x/jx46p2kn29bbg4k41vh6c4gh0000gn/T/ipykernel_16616/1544277031.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


In [12]:
print(full_name)
print(annual_income-annual_expenses)

Jacob
50000


In [ ]:
# Load env

In [ ]:
# Use questions to determine the clients risk tolerance, investment horizen, income v. growth   

In [16]:
# Convert Investor Responses to Risk Scores

risk_score = 0
time_score = 0

investment_percent = investment_amount / annual_income
disposible_income = annual_income - annual_expenses
investment_ratio = disposible_income / investment_amount

if investment_percent < .40:
    risk_score += 100
elif investment_percent > .40 and investment_percent < .80:
    risk_score += 50
else:
    risk_score += 0

if investing_experience < 4:
    risk_score -= 10
elif investing_experience > 4 and investing_experience < 9:
    risk_score += 0
else:
    risk_score += 10

if income_stability:
    risk_score += 0
else:
    risk_score -= 40

if risk_level == "Low":
    risk_score * .1
elif risk_level == "Moderate":
    risk_score * .5
elif risk_level == "High":
    risk_score * 1
else:
    risk_score * 1.10

if investment_length == "0-1":
    time_score += 25
elif investment_length == "2-4" and investment_length <= 4:
    time_score += 50
elif investment_length == "5-9" and investment_length <= 9:
    time_score += 75
else:
    time_score += 100

In [17]:
print(risk_score)
print(time_score)
print(investment_strategy)

110
25
Value


In [11]:
# Import stock symbol db
nasdaq = pd.read_csv(
	Path("Resources/nasdaq.csv"),
	index_col = "Symbol",
)

display(nasdaq.head())
display(nasdaq.tail())



,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
Symbol,,,,,,,,,,
A,Agilent Technologies Inc. Common Stock,$141.12,-0.67,-0.473%,4.261835e+10,United States,1999.0,1087377,Capital Goods,Electrical Products
AA,Alcoa Corporation Common Stock,$64.22,1.48,2.359%,1.182289e+10,NaN,2016.0,7159515,Basic Industries,Metal Fabrications
AAC,Ares Acquisition Corporation Class A Ordinary ...,$9.75,-0.01,-0.102%,1.218750e+09,NaN,2021.0,49771,Finance,Business Services
AACG,ATA Creativity Global American Depositary Shares,$1.56,0.02,1.299%,4.894800e+07,China,NaN,61790,Miscellaneous,Service to the Health Industry
AACI,Armada Acquisition Corp. I Common Stock,$9.8229,0.00,0.00%,2.034273e+08,United States,2021.0,6,NaN,NaN


,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
Symbol,,,,,,,,,,
ZWS,Zurn Water Solutions Corporation Common Stock,$30.76,-0.04,-0.13%,3.867149e+09,United States,2012.0,1304547,Public Utilities,Water Supply
ZY,Zymergen Inc. Common Stock,$4.43,0.16,3.747%,4.536355e+08,United States,2021.0,788529,Basic Industries,Industrial Specialties
ZYME,Zymeworks Inc. Common Shares,$7.95,0.46,6.142%,4.578323e+08,Canada,2017.0,1132143,NaN,NaN
ZYNE,Zynerba Pharmaceuticals Inc. Common Stock,$2.67,0.07,2.692%,1.100508e+08,United States,2015.0,273878,Health Care,Biotechnology: Pharmaceutical Preparations
ZYXI,Zynex Inc. Common Stock,$7.52,0.23,3.155%,3.015435e+08,United States,NaN,154611,Health Care,Other Pharmaceuticals


In [12]:
# Clean NASDAQ Data by Removing Columns

nasdaq = nasdaq.drop(['Last Sale', 'Net Change', '% Change', 'IPO Year', 'Country'], axis=1)

nasdaq.head()

,Name,Market Cap,Volume,Sector,Industry
Symbol,,,,,
A,Agilent Technologies Inc. Common Stock,4.261835e+10,1087377,Capital Goods,Electrical Products
AA,Alcoa Corporation Common Stock,1.182289e+10,7159515,Basic Industries,Metal Fabrications
AAC,Ares Acquisition Corporation Class A Ordinary ...,1.218750e+09,49771,Finance,Business Services
AACG,ATA Creativity Global American Depositary Shares,4.894800e+07,61790,Miscellaneous,Service to the Health Industry
AACI,Armada Acquisition Corp. I Common Stock,2.034273e+08,6,NaN,NaN


In [ ]:
# Load stocks based on client suitability

#  If variables to select the etfs

In [ ]:
# Determine info for SPY, QQQ, IEF, and client selections

# Use Alpaca to pull last 3 years data for SPY, QQQ, IEF, and Client Selections

In [ ]:
# Run MC Sim for SPY, DIA, IEF, and client selection

In [ ]:
# Present comparisons in table

In [ ]:
# Present comparisons graphically

In [ ]:
# Determine whether the client wants a PDF of the report

In [6]:
# Provide written report and recommendations

pdf = FPDF('P', 'mm', 'letter')
pdf.add_page()
pdf.set_font('Arial', 'B', 16)
pdf.cell(40, 10, 'Report to begin')
pdf.output('portfolio_suitability_report.pdf', 'F')


''

In [11]:
# Mail the document
# mail = Mailer(email='portfoliosuitability@gmail.com', password='RiceBootCamp2022')
# mail.send(receiver='cdb@bcg.law', subject='You made it work!', message='Congrats')

TypeError: __init__() got an unexpected keyword argument 'email'

In [ ]:

# # run the questions using fire.Fire
# def run():
   

#    survey_questions()
   
   

# if __name__ == "__main__":
#     fire.Fire(run)